# Anthropic Functions & Tools Example

### Install Dependencies

In [1]:
%pip install anthropic
%pip install defusedxml
%pip install requests


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import Libraries

In [2]:
import sys
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
from defusedxml import ElementTree
from collections import defaultdict
import os
from typing import Any
import requests

### Setup Parameters

In [21]:
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
user_input = "Whats the difference between the price of Ethereum and Maker?"
model = "claude-2.1"

### Define Tools

In [5]:
def get_weather(latitude: str, longitude: str):
  url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
  response = requests.get(url)
  return response.json()

In [6]:
def get_lat_long(place):
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params).json()
    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None

In [22]:
def get_crypto_price(name):
    url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"

    params = {'ids': name.lower(), 'vs_currencies': 'usd'}
    response = requests.get(url, params=params).json()

    if response:
        return response
    else:
        return None

In [7]:
get_weather_description = """
<tool_description>
<tool_name>get_weather</tool_name>
<description>
Returns weather data for a given latitude and longitude. </description>
<parameters>
<parameter>
<name>latitude</name>
<type>string</type>
<description>The latitude coordinate as a string</description>
</parameter> <parameter>
<name>longitude</name>
<type>string</type>
<description>The longitude coordinate as a string</description>
</parameter>
</parameters>
</tool_description>
"""

In [8]:
get_lat_long_description = """<tool_description>
<tool_name>get_lat_long</tool_name>
<description>
Returns the latitude and longitude for a given place name.
</description>
<parameters>
<parameter>
<name>place</name>  
<type>string</type>
<description>
The place name to geocode and get coordinates for.
</description>
</parameter>
</parameters>
</tool_description>"""

In [23]:
get_crypto_price_description = """<tool_description>
<tool_name>get_crypto_price</tool_name>
<description>
Returns the price of a cryptocurrency in usd.
</description>
<parameters>
<parameter>
<name>name</name>  
<type>string</type>
<description>
The full name of the cryptocurrency in lowercase.
</description>
</parameter>
</parameters>
</tool_description>"""

In [24]:
list_of_tools_specs = [get_weather_description, get_lat_long_description, get_crypto_price_description]

tools_string = ""
for tool_spec in list_of_tools_specs:
    tools_string += tool_spec

### Create Tool Use Prompt

In [10]:
def create_prompt(user_input):
    prompt_template = f"""
In this environment, you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

Human:
{user_input}


Assistant:
"""
    return prompt_template

### Call Function

In [11]:
def call_function(tool_name, parameters):
    func = globals()[tool_name] 
    output = func(**parameters)
    return output

### Format Result

This function will format the result from our function call into the proper XML structure.

In [12]:
def format_result(tool_name, output):
    return f"""
<function_results>
<result>
<tool_name>{tool_name}</tool_name>
<stdout>
{output}
</stdout>
</result>
</function_results>
"""

### XML Etree to Python Dict

In [19]:
def etree_to_dict(t) -> dict[str, Any]:
    d = {t.tag: {}}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(("@" + k, v) for k, v in t.attrib.items())
    if t.text and t.text.strip():
        if children or t.attrib:
            d[t.tag]["#text"] = t.text
        else:
            d[t.tag] = t.text
    return d

### Create Run Loop Logic

This function is the actual orchestrator of the function calling logic. Here's how it works:

1. We kick off a loop that first calls Claude with our tool use prompt with the tool specs and the user input loaded into it.
2. We get the completion from Claude and check if the stop sequence for the completion was the closing XML tag for a function call, ```</function_calls>```
3. If the completion does in fact contain a function call, we extract out the tool name and the tool parameters from the XML.
4. We then call the function that Claude has decided to invoke.
5. We take the results of the function call, format them into an XML structure, and add them back to the prompt.
6. We repeat the loop starting at step 1 with the original prompt plus the text that has been appended.
7. This process continues until Claude finally outputs an answer and we break the loop.

In [15]:
anthropic = Anthropic(api_key=ANTHROPIC_API_KEY)
def run_loop(prompt):
    print(prompt)
    # Start function calling loop
    while True:
        # Get a completion from Claude
        partial_completion = anthropic.completions.create(prompt = prompt,
                                                        stop_sequences=["\n\nHuman:", "</function_calls>"],
                                                        model="claude-2.1",
                                                        max_tokens_to_sample = 1000,
                                                        temperature = 0)
        partial_completion, stop_reason, stop_seq = partial_completion.completion, partial_completion.stop_reason, partial_completion.stop # type: ignore

        # Append the completion to the end of the prompt
        prompt += partial_completion
        print(partial_completion)

        if stop_reason == 'stop_sequence' and stop_seq == '</function_calls>':
            start_index = partial_completion.find("<function_calls>")
            if start_index != -1:
                # Extract the XML Claude outputted (invoking the function)
                extracted_text = partial_completion[start_index+16:]

                # Parse the XML find the tool name and the parameters that we need to pass to the tool
                xml = ElementTree.fromstring(extracted_text)
                tool_name_element = xml.find("tool_name")

                if tool_name_element is None:
                    print("Unable to parse function call, invalid XML or missing 'tool_name' tag")
                    break

                tool_name_from_xml = tool_name_element.text.strip()
                parameters_xml = xml.find("parameters")
                
                if parameters_xml is None:
                    print("Unable to parse function call, invalid XML or missing 'parameters' tag")
                    break

                param_dict = etree_to_dict(parameters_xml)
                parameters = param_dict["parameters"]

                output = call_function(tool_name_from_xml, parameters)

                prompt += "</function_calls>"
                print("</function_calls>")

                # Add the result from calling the tool back to the prompt
                function_result = format_result(tool_name_from_xml, output)
                prompt += function_result
                print(function_result)

        else:
            break

### Make Request

In [25]:
prompt = create_prompt(user_input)

In [26]:
print(prompt)


In this environment, you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

<tool_description>
<tool_name>get_weather</tool_name>
<description>
Returns weather data for a given latitude and longitude. </description>
<parameters>
<parameter>
<name>latitude</name>
<type>string</type>
<description>The latitude coordinate as a string</description>
</parameter> <parameter>
<name>longitude</name>
<type>string</type>
<description>The longitude coordinate as a string</description>
</parameter>
</parameters>
</tool_description>
<tool_description>
<tool_name>get_lat_long</tool_name>
<description>
Returns the latitude and longitude for a given p

In [27]:
run_loop(prompt)


In this environment, you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

<tool_description>
<tool_name>get_weather</tool_name>
<description>
Returns weather data for a given latitude and longitude. </description>
<parameters>
<parameter>
<name>latitude</name>
<type>string</type>
<description>The latitude coordinate as a string</description>
</parameter> <parameter>
<name>longitude</name>
<type>string</type>
<description>The longitude coordinate as a string</description>
</parameter>
</parameters>
</tool_description>
<tool_description>
<tool_name>get_lat_long</tool_name>
<description>
Returns the latitude and longitude for a given p